<a href="https://colab.research.google.com/github/fabiobasson/Bi-Master/blob/main/geological_image_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from os import getcwd
import zipfile
import shutil
import numpy as np
import glob
import random
import pandas as pd
import matplotlib.pyplot as plt


from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
!rm -rf geological_similarity

**Importar arquivos de configuração do Kaglle**

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"fabiobasson","key":"d71cbfdfd23379909173204eac74cf96"}'}

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

###  Download dos dados / descompactando os dados

In [6]:
!kaggle datasets download fabiobasson/geological-similarity
! unzip -qq  geological-similarity

  0% 0.00/1.99M [00:00<?, ?B/s]
100% 1.99M/1.99M [00:00<00:00, 66.4MB/s]


In [5]:
! rm -rf gneiss andesite

### Data Preprocessing

In [7]:
andesite_dir = glob.glob('andesite/*.jpg')
gneiss_dir = glob.glob('gneiss/*.jpg')
marble_dir = glob.glob('geological_similarity/marble/*.jpg');quartzite_dir = glob.glob('geological_similarity/quartzite/*.jpg')
rhyolite_dir = glob.glob('geological_similarity/rhyolite/*.jpg');schist_dir = glob.glob('geological_similarity/schist/*.jpg')
print(len(andesite_dir));print(len(gneiss_dir)); print(len(marble_dir)); print(len(quartzite_dir));  print(len(schist_dir))


1008
948
0
0
0


In [13]:
andesite_dir[0:4]

['andesite/9FUK8.jpg',
 'andesite/3VFJB.jpg',
 'andesite/7YPFT.jpg',
 'andesite/3PTNB.jpg']

In [ ]:
#geological_similarity = 'geological_similarity'
#subfolders = [f.path for f in os.scandir(geological_similarity) if f.is_dir()]


In [8]:
class_names = ['andesite', 'gneiss', 'marbler', 'quartzite', 'rhyolite',
               'schist']
class_names[0]

'andesite'

In [14]:
for i in andesite_dir[0:5]:
    andesite_dir.append([i,class_names[0]])
for j in gneiss_dir[0:5]:
   gneiss_dir.append([j,class_names[1]])
df = andesite_dir + gneiss_dir
random.shuffle(df)
df

['andesite/05KA0.jpg',
 'gneiss/7VHIN.jpg',
 'andesite/7IQJZ.jpg',
 'gneiss/3D6R4.jpg',
 'andesite/03GPU.jpg',
 'gneiss/3HVJ7.jpg',
 'andesite/0YCVK.jpg',
 'gneiss/4BEHO.jpg',
 'andesite/5RT4L.jpg',
 'andesite/6FW0S.jpg',
 'andesite/0T3EK.jpg',
 'andesite/2E65R.jpg',
 'andesite/8SLQP.jpg',
 'andesite/3UDH8.jpg',
 'andesite/5IFOB.jpg',
 'gneiss/4DRTY.jpg',
 'andesite/0DHSF.jpg',
 'gneiss/5KSAC.jpg',
 'gneiss/2LAB5.jpg',
 'andesite/4O4RL.jpg',
 'andesite/0TLMK.jpg',
 'andesite/8T5BJ.jpg',
 'gneiss/7KLZX.jpg',
 'gneiss/4NEP8.jpg',
 'gneiss/6NRHC.jpg',
 'andesite/5W84L.jpg',
 'gneiss/7P8GB.jpg',
 'andesite/2SY97.jpg',
 'gneiss/6S04D.jpg',
 'andesite/5MC2B.jpg',
 'gneiss/7K6YM.jpg',
 'andesite/4LBM7.jpg',
 'andesite/6CBI0.jpg',
 ['gneiss/0ICYJ.jpg', 'gneiss'],
 'gneiss/8Z3W4.jpg',
 'gneiss/4WZTA.jpg',
 'andesite/1TMTT.jpg',
 'gneiss/7YWDC.jpg',
 'gneiss/7SNA1.jpg',
 'andesite/6H383.jpg',
 'andesite/3BZ2V.jpg',
 'gneiss/4S3TZ.jpg',
 'gneiss/0FGWI.jpg',
 'gneiss/4WNU8.jpg',
 'andesite/4AXA4.j

In [19]:
data_df = pd.DataFrame(df,columns=['path'])
data_df

,path
0,andesite/05KA0.jpg
1,gneiss/7VHIN.jpg
2,andesite/7IQJZ.jpg
3,gneiss/3D6R4.jpg
4,andesite/03GPU.jpg
...,...
1961,andesite/1ZE35.jpg
1962,gneiss/3YFZ5.jpg
1963,andesite/2JWAF.jpg
1964,gneiss/6L0EW.jpg


In [79]:
data_df.columns

RangeIndex(start=0, stop=18, step=1)